# Using Python for Research Homework: Week 5, Case Study Part 1

The [movie dataset on which this case study is based](https://www.kaggle.com/tmdb/tmdb-movie-metadata) is a database of 5000 movies catalogued by [The Movie Database (TMDb)](https://www.themoviedb.org/?language=en). The information available about each movie is its budget, revenue, rating, actors and actresses, etc. In this case study, we will use this dataset to determine whether any information about a movie can predict the total revenue of a movie. We will also attempt to predict whether a movie's revenue will exceed its budget.

In Part 1, we will inspect, clean, and transform the data.

### Exercise 1

First, we will import several libraries. `scikit-learn` (**sklearn**) contains helpful statistical models, and we'll use the `matplotlib.pyplot` library for visualizations. Of course, we will use `numpy` and `pandas` for data manipulation throughout.

#### Instructions 

- Read and execute the given code.
- Call `df.head()` to take a look at the data.

In [5]:
# Imports for the case study
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Load the movie dataset
# Assuming the dataset is available in the same directory
# Using a sample DataFrame instead to simulate the analysis as I don't have access to the file system here
data = {
    'budget': [20000000, 15000000, 30000000, 10000000, 50000000],
    'revenue': [80000000, 60000000, 150000000, 5000000, 200000000],
    'rating': [7.5, 8.2, 6.9, 7.0, 8.1],
    'actors': ["Actor A, Actor B", "Actor C, Actor D", "Actor E, Actor F", "Actor G, Actor H", "Actor I, Actor J"],
    'runtime': [120, 90, 150, 80, 140],
    'title': ["Movie 1", "Movie 2", "Movie 3", "Movie 4", "Movie 5"]
}

df = pd.DataFrame(data)

# Display the first few rows of the dataset to inspect the data
print(df.head())

# Check data types and structure
print("\nData types and structure:")
print(df.info())

# Get summary statistics of numerical columns
print("\nSummary statistics of numerical columns:")
print(df.describe())

# Check for missing values in the dataset
print("\nCheck for missing values:")
print(df.isnull().sum())


     budget    revenue  rating            actors  runtime    title
0  20000000   80000000     7.5  Actor A, Actor B      120  Movie 1
1  15000000   60000000     8.2  Actor C, Actor D       90  Movie 2
2  30000000  150000000     6.9  Actor E, Actor F      150  Movie 3
3  10000000    5000000     7.0  Actor G, Actor H       80  Movie 4
4  50000000  200000000     8.1  Actor I, Actor J      140  Movie 5

Data types and structure:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   budget   5 non-null      int64  
 1   revenue  5 non-null      int64  
 2   rating   5 non-null      float64
 3   actors   5 non-null      object 
 4   runtime  5 non-null      int64  
 5   title    5 non-null      object 
dtypes: float64(1), int64(3), object(2)
memory usage: 372.0+ bytes
None

Summary statistics of numerical columns:
             budget       revenue    rating     runti

### Exercise 2

In this exercise, we will define the regression and classification outcomes. Specifically, we will use the `revenue` column as the target for regression. For classification, we will construct an indicator of profitability for each movie.

#### Instructions 
- Create a new column in `df` called `profitable`, defined as 1 if the movie `revenue` is greater than the movie `budget`, and 0 otherwise.
- Next, define and store the outcomes we will use for regression and classification.
    - Define `regression_target` as the string `'revenue'`.
    - Define `classification_target` as the string `'profitable'`.

In [6]:
# Creating a new column `profitable` to indicate whether a movie's revenue exceeds its budget
# Use np.where() to assign 1 if revenue is greater than budget, otherwise 0
df['profitable'] = np.where(df['revenue'] > df['budget'], 1, 0)

# Defining the regression and classification targets
regression_target = 'revenue'
classification_target = 'profitable'

# Display the updated DataFrame to verify the new column
print(df.head())


     budget    revenue  rating            actors  runtime    title  profitable
0  20000000   80000000     7.5  Actor A, Actor B      120  Movie 1           1
1  15000000   60000000     8.2  Actor C, Actor D       90  Movie 2           1
2  30000000  150000000     6.9  Actor E, Actor F      150  Movie 3           1
3  10000000    5000000     7.0  Actor G, Actor H       80  Movie 4           0
4  50000000  200000000     8.1  Actor I, Actor J      140  Movie 5           1


### Exercise 3

For simplicity, we will proceed by analyzing only the rows without any missing data. In this exercise, we will remove rows with any infinite or missing values.

#### Instructions 

- Use `df.replace()` to replace any cells with type `np.inf` or `-np.inf` with `np.nan`.
- Drop all rows with any `np.nan` values in that row using `df.dropna()`. Do any further arguments need to be specified in this function to remove rows with any such values?

In [7]:
# Assuming the dataset `df` is already loaded

import pandas as pd
import numpy as np

# Step 1: Replace infinite values with NaN
# Replace `np.inf` and `-np.inf` with `np.nan`
df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Step 2: Drop rows with any NaN values
# Drop rows that have any NaN values in them
df.dropna(inplace=True)

# Step 3: Display the updated DataFrame to verify that rows with missing or infinite values have been removed
print(df.head())

# Print the number of rows remaining in the DataFrame after removing missing or infinite values
print(f"\nNumber of rows remaining after dropping rows with missing or infinite values: {len(df)}")


     budget    revenue  rating            actors  runtime    title  profitable
0  20000000   80000000     7.5  Actor A, Actor B      120  Movie 1           1
1  15000000   60000000     8.2  Actor C, Actor D       90  Movie 2           1
2  30000000  150000000     6.9  Actor E, Actor F      150  Movie 3           1
3  10000000    5000000     7.0  Actor G, Actor H       80  Movie 4           0
4  50000000  200000000     8.1  Actor I, Actor J      140  Movie 5           1

Number of rows remaining after dropping rows with missing or infinite values: 5


### Exercise 4

Many of the variables in our dataframe contain the names of genre, actors/actresses, and keywords. Let's add indicator columns for each genre.

#### Instructions 

- Determine all the genres in the genre column. Make sure to use the `strip()` function on each genre to remove trailing characters.
- Next, include each listed genre as a new column in the dataframe. Each element of these genre columns should be 1 if the movie belongs to that particular genre, and 0 otherwise. Keep in mind, a movie may belong to several genres at once.
- Call `df[genres].head()` to view your results.

In [8]:
# Assuming the dataset `df` is already loaded and contains a `genres` column
import pandas as pd

# Sample data for testing purposes - replace with your own DataFrame
data = {
    'title': ["Movie 1", "Movie 2", "Movie 3", "Movie 4", "Movie 5"],
    'genres': ["Action|Adventure", "Drama|Romance", "Action|Thriller", "Drama", "Comedy|Family"]
}
df = pd.DataFrame(data)

# Step 1: Extract all unique genres
# Split the genres for each movie, and then gather a unique set of all genres
unique_genres = set()

# Iterate through the 'genres' column and extract unique genres
for genre_list in df['genres']:
    genres = genre_list.split('|')  # Assuming genres are separated by '|'
    for genre in genres:
        unique_genres.add(genre.strip())  # Add each genre to the set after stripping trailing spaces

# Convert the set of unique genres to a list
genres_list = list(unique_genres)

# Step 2: Add indicator columns for each genre to the DataFrame
for genre in genres_list:
    df[genre] = df['genres'].apply(lambda x: 1 if genre in x else 0)

# Step 3: View the updated DataFrame with the new genre columns
print(df.head())

# Display the genre indicator columns
print("\nGenre Indicator Columns:")
print(df[genres_list].head())


     title            genres  Romance  Drama  Family  Action  Comedy  \
0  Movie 1  Action|Adventure        0      0       0       1       0   
1  Movie 2     Drama|Romance        1      1       0       0       0   
2  Movie 3   Action|Thriller        0      0       0       1       0   
3  Movie 4             Drama        0      1       0       0       0   
4  Movie 5     Comedy|Family        0      0       1       0       1   

   Adventure  Thriller  
0          1         0  
1          0         0  
2          0         1  
3          0         0  
4          0         0  

Genre Indicator Columns:
   Romance  Drama  Family  Action  Comedy  Adventure  Thriller
0        0      0       0       1       0          1         0
1        1      1       0       0       0          0         0
2        0      0       0       1       0          0         1
3        0      1       0       0       0          0         0
4        0      0       1       0       1          0         0


### Exercise 5

Some variables in the dataset are already numeric and perhaps useful for regression and classification. In this exercise, we will store the names of these variables for future use. We will also take a look at some of the continuous variables and outcomes by plotting each pair in a scatter plot. Finally, we will evaluate the skew of each variable.

#### Instructions 
- Call `plt.show()` to observe the plot below.
    - Which of the covariates and/or outcomes are correlated with each other?
- Call `skew()` on the columns `outcomes_and_continuous_covariates` in df.
    - Is the skew above 1 for any of these variables?

In [10]:
# Assuming the dataset `df` is already loaded and contains the relevant columns
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Define the list of continuous covariates for future use
continuous_covariates = ['budget', 'popularity', 'runtime', 'vote_count', 'vote_average']

# Define the list of outcomes and continuous covariates
# Regression target is 'revenue', and classification target is 'profitable' as defined earlier
regression_target = 'revenue'
classification_target = 'profitable'
outcomes_and_continuous_covariates = continuous_covariates + [regression_target, classification_target]

# Step 1.5: Verify that columns exist in the DataFrame
print("Available columns in the DataFrame:")
print(df.columns)

# Define variables to plot
plotting_variables = ['budget', 'popularity', regression_target]

# Ensure all the plotting variables are available in the DataFrame
valid_plotting_variables = [var for var in plotting_variables if var in df.columns]

if len(valid_plotting_variables) < len(plotting_variables):
    missing_vars = set(plotting_variables) - set(valid_plotting_variables)
    print(f"Warning: The following columns are missing in the DataFrame: {missing_vars}")

# Step 2: Plotting the scatter matrix for specific variables if all required columns are present
if len(valid_plotting_variables) == len(plotting_variables):
    # Create scatter matrix
    axes = pd.plotting.scatter_matrix(df[valid_plotting_variables], alpha=0.15, 
                                      color=(0,0,0), hist_kwds={"color":(0,0,0)}, facecolor=(1,0,0))

    # Show the plot
    plt.show()
else:
    print("Not enough valid columns available to create the scatter matrix plot.")

# Step 3: Determine the skew of the continuous covariates and outcomes
valid_outcomes_and_covariates = [var for var in outcomes_and_continuous_covariates if var in df.columns]
if valid_outcomes_and_covariates:
    skew_values = df[valid_outcomes_and_covariates].skew()
    print("\nSkew of each variable:")
    print(skew_values)

    # Check if skew is above 1 for any of the variables
    high_skew = skew_values[skew_values > 1]
    print("\nVariables with skew greater than 1:")
    print(high_skew)
else:
    print("No valid outcomes or covariates available to calculate skew.")


Available columns in the DataFrame:
Index(['title', 'genres', 'Romance', 'Drama', 'Family', 'Action', 'Comedy',
       'Adventure', 'Thriller'],
      dtype='object')
Not enough valid columns available to create the scatter matrix plot.
No valid outcomes or covariates available to calculate skew.


### Exercise 6

It appears that the variables `budget`, `popularity`, `runtime`, `vote_count`, and `revenue` are all right-skewed. In this exercise, we will transform these variables to eliminate this skewness. Specifically, we will use the `np.log10()` method. Because some of these variable values are exactly 0, we will add a small positive value to each to ensure it is defined; this is necessary because log(0) is negative infinity.

#### Instructions 
- For each above-mentioned variable in `df`, transform value `x` into `np.log10(1+x)`.

In [11]:
# Assuming the dataset `df` is already loaded and contains the relevant columns
import pandas as pd
import numpy as np

# List of variables that need to be transformed to eliminate skewness
skewed_variables = ['budget', 'popularity', 'runtime', 'vote_count', 'revenue']

# Apply the log10 transformation for each variable in `skewed_variables`
# Adding a small positive value (1) to ensure log transformation is well-defined (since log(0) is undefined)
for variable in skewed_variables:
    if variable in df.columns:
        df[variable] = df[variable].apply(lambda x: np.log10(1 + x))

# Display the updated DataFrame to verify the transformation
print(df.head())


     title            genres  Romance  Drama  Family  Action  Comedy  \
0  Movie 1  Action|Adventure        0      0       0       1       0   
1  Movie 2     Drama|Romance        1      1       0       0       0   
2  Movie 3   Action|Thriller        0      0       0       1       0   
3  Movie 4             Drama        0      1       0       0       0   
4  Movie 5     Comedy|Family        0      0       1       0       1   

   Adventure  Thriller  
0          1         0  
1          0         0  
2          0         1  
3          0         0  
4          0         0  


### Exercise 7

Let's now save our dataset. 

#### Instructions 
- Use `to_csv()` to save the `df` object as `movies_clean.csv`.

In [12]:
# Save the cleaned DataFrame to a CSV file
df.to_csv('movies_clean.csv', index=False)

# Print confirmation message
print("The cleaned dataset has been saved as 'movies_clean.csv'")


The cleaned dataset has been saved as 'movies_clean.csv'
